In [2]:
from pathlib import Path
import pandas as pd

In [12]:
results_file = Path('/Users/djohnson/projects/evalxai_studies/apps/results_validation.csv')
df = pd.read_csv(results_file)
df.sample(5)

,id,true,pred,select
20,4b1c108d-8b62-4e1e-b774-cc923a888799,1,1,1
11,41002a17-0b92-4d5c-ad0e-dd6b195c397c,1,0,1
30,768159d9-8da4-4004-8ca5-b40a0969ae53,0,0,0
24,ea62e53b-3060-4c01-80ba-a681afbe9884,1,1,1
1,ff70b4d2-a056-486c-8e44-5ceefa4d113f,0,1,0


In [13]:
part_acc = (df['true'] == df['select']).astype(int).mean()
model_acc = (df['true'] == df['pred']).astype(int).mean()

part_model = (df['pred'] == df['select']).astype(int).mean()

print(f'Model Acc: {model_acc}')
print(f'Part Acc: {part_acc}')
print(f'Part matches model: {part_model}')

Model Acc: 0.6666666666666666
Part Acc: 1.0
Part matches model: 0.6666666666666666


In [14]:
or_cond = (df['pred'] == df['select']) & (df['pred'] != df['true'])
overreliance = (or_cond).astype(int).sum() / (df['pred'] != df['true']).sum()
overreliance

0.0

In [15]:
df[or_cond]

,id,true,pred,select
